# Bottle Cap Color Detection – Experimental Notebook

Notebook ini berisi seluruh langkah eksplorasi, relabeling, training, evaluasi, dan analisis untuk model pendeteksi tutup botol 3 warna.


In [ ]:
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

from bsort.config import load_config

cfg = load_config("../settings.yaml")
cfg


## Tujuan Notebook

1. Memahami dataset
2. Melakukan relabel warna (light_blue, dark_blue, others)
3. Menjalankan training
4. Mengevaluasi model
5. Mengukur inference time
6. Menulis kesimpulan

Notebook ini fokus pada *penjelasan dan analisis*, bukan implementasi pipeline, karena pipeline sudah ada di `src/bsort`.


In [ ]:
train_images = Path(cfg.dataset.train_images)
sample = next(train_images.glob("*.jpg"))
img = cv2.imread(str(sample))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
sample


## Relabeling Strategy (HSV)

Dataset asli tidak memiliki label warna.  
Jadi setiap bounding box akan diambil patch-nya → dihitung rata-rata HSV → ditentukan warnanya:

- Jika Hue di rentang biru + Value tinggi → **light_blue**
- Jika Hue di rentang biru + Value rendah → **dark_blue**
- Selain itu → **others**


In [ ]:
def get_avg_hsv(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = hsv.mean(axis=(0,1))
    return h, s, v

get_avg_hsv(img)


In [ ]:
# contoh (kalau sudah bikin relabel di data.py)
# from bsort.data import relabel_split
# relabel_split(Path(cfg.dataset.train_images), Path(cfg.dataset.train_labels))
# relabel_split(Path(cfg.dataset.val_images), Path(cfg.dataset.val_labels))

print("Relabel done (gunakan script data.py)")


## Training Model

Training tidak dilakukan di notebook agar pipeline tetap rapi.
Gunakan CLI:



In [ ]:
!bsort train --config ../settings.yaml


In [ ]:
model = YOLO(cfg.model.best_model_path)
metrics = model.val(data="artifacts/data_bottlecap.yaml")
metrics


In [ ]:
val_img = next(Path(cfg.dataset.val_images).glob("*.jpg"))
result = model(val_img)[0]
plt.imshow(result.plot()[:,:,::-1])
plt.axis("off")
plt.title("Prediction Example")


In [ ]:
import time

def bench(model, img, n=50):
    model(img, verbose=False)
    start = time.perf_counter()
    for _ in range(n):
        model(img, verbose=False)
    end = time.perf_counter()
    return (end - start)*1000/n

img = cv2.imread(str(val_img))
bench(model, img, 50)


import time

def bench(model, img, n=50):
    model(img, verbose=False)
    start = time.perf_counter()
    for _ in range(n):
        model(img, verbose=False)
    end = time.perf_counter()
    return (end - start)*1000/n

img = cv2.imread(str(val_img))
bench(model, img, 50)
